# TN Districts

## Progress

The Goobers are still grappling with step #1

## The Imports

In [35]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
from datetime import datetime
from lxml import html
import requests
import re

## The Soup Prep

In [53]:
# variables to pass into the opensecrets url.
State = 'Tn'
District = '07'
#opensecrets url with state and district variables passed.
#note: the website will give data if the conventions used are incorrect. !!!USE EXTREME CAUTION!!!
#eg. if you use `7` instead of `07` for districts, it won't throw an error. I'll take you to the wrong page.
#tried dumping all district data into one soup object. Wanted to iterate `01` - `07`. The soup doesn't like it.
#tried defining district variable as `0o7` instead of string. I though it would make the soup happy. It did not.
url = "https://www.opensecrets.org/races/candidates?cycle=2020&id="+State+District+"&spec=N"
page = requests.get(url)
# variables to make soup objects
soup = BS(page.content, 'html.parser')
persons = soup.findAll('div',  {'class':"Members--list-item"})
# money isn't correct. The soup won't give up its secrets.
money = soup.findAll('table', {'class':"Members--list-item"})
# usless, near as I can tell.  Will keep incase it proves useful.
tbl = soup.find("table",{"id":"MainContent_dataGridView1"})

In [55]:
#The soup's title
#rerun this cell if changing district/state variables from cell above.
soup.title

<title>Tennessee District 07 2020 Race • OpenSecrets</title>

In [57]:
#The soup will not give me money.
print(money)

[]


## The Soup Functions

In [60]:
# list of functions to find chunks of soup.
# This is a combination of several cells from previous team collaboration.
# lots of repeated code. Want to clean soup, not broken soup.
person = persons # removed '[3]' because it was breaking code for other districts. Soup doesn't mind.
#Function to find name of candidate
def get_name (person):
    member_text = person.find("h2").text
    member_text = member_text.replace("\t", "").replace("\n", "")
    return re.search(r"(.+) \(.\)", member_text)[1]
#Function to find candidate's political affiliation
def get_party (person):
    member_text = person.find("h2").text
    member_text = member_text.replace("\t", "").replace("\n", "")
    return re.search(r"\((.)\)", member_text)[1]
#Function to find total number of votes each
def votes (person):
    member_text = person.find("h2").text
    member_text = member_text.replace("\t", "").replace("\n", "")
    return re.search("\((\d+\.\d)\%", member_text)[1]

# "person" soup as cooked breaks down after this point.
# money raised and money spent are in "Members--table"
# information of interest is contained in "<td>" tags.

#Missing Functions and associated lists

#def state:
#def district:
#def incumbant:
#def winner:
#def money_raised(person):
#def money_spent(person):

## The List Loops

In [47]:
# for loops to pour soup functions into lists
# This is a combination of several cells from previous team collaboration.
# This could also be cleaned up. list comprehension?

#Loop for name list
name_list = []
for person in persons:
     name_list.append(get_name (person))
#Loop for party affiliation list
party_list = []
for person in persons:
    party_list.append(get_party (person))
#Loop for total votes recieved list
    votes_received = []
for person in persons:
    votes_received.append(votes (person))
# once defined, functions for money raised, money spent, state, district, incumbant, and winner go here.

## The Pandas Food
### (╯°□°)╯︵ ┻━┻

In [49]:
# not behaving as I would expect.
# I'm misunderstanding the append function.
# I wanted to create lists, iterate through the url, make a new soup, then appended to data already in list.
# When I do this, I capture the new district's data, but the previous district's lists are overwritten.
pd.DataFrame({"Name": name_list, "Party": party_list, "Percent_Vote": votes_received})

,Name,Party,Percent_Vote
0,Mark Green,R,69.9
1,Kiran Sreepada,D,27.3
2,Ronald Brown,I,2.2
3,Scott Vieira Jr,I,0.6


# An Analysis of Political Contributions During the 2020 House of Representatives Election

In this part, you will obtain as much data as you can on the campaign contributions received by each candidate. This data is avaiable through the website https://www.opensecrets.org/.

1. Start by scraping the data from the summary page for Tennessee's 7th District, which is available at https://www.opensecrets.org/races/candidates?cycle=2020&id=TN07&spec=N.
   

* Make a DataFrame showing, for each candidate:
    * the candidate's name
    * the candidate's party
    * state
    * district number
    * whether the candidate was an incumbent
    * whether the candidate won the race
    * the percentage of the vote that candidate received
    * the total amount raised by that candidate (as a numeric variable)
    * the total amount spent by the candidate (as a numeric variable)

2. Once you have working code for Tennessee's 7th District, expand on your code to capture all of Tennessee's districts.

3. Once you have working code for all of Tennessee's districts, expand on it to capture all states and districts. The number of representatives each state has can be found in a table on this page: https://www.britannica.com/topic/United-States-House-of-Representatives-Seats-by-State-1787120

4. Using your scraped data, investigates different relationships between candidates and the amount of money they raised. Here are some suggestions to get you started, but feel free to pose you own questions or do additional exploration:  
    a. How often does the candidate who raised more money win a race?  
    b. How often does the candidate who spent more money win a race?  
    c. Does the difference between either money raised or money spent seem to influence the likelihood of a candidate winning a race?  
    d. How often does the incumbent candidate win a race?  
    e. Can you detect any relationship between amount of money raised and the incumbent status of a candidate?

### Bonus Questions:
If you complete all of the above, you can attempt these challenging bonus questions.

Open Secrets also gives a detailed breakdown of contributions by source.

Scrape these pages to get information on contributions by source. See if you can find anything interesting in terms of the source of contributions. Some examples to get you started:
* What does the overall distribution of funding sources look like?
* Is there any detectable difference in contribution sources between Democrat and Republican candidates?
* Do the funding sources for either the winning candidate or incumbent candidate differ from the other candidates?